<a href="https://colab.research.google.com/github/brianpyai/chatbot/blob/main/gemma_playgrounds_GGUF_4Bit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install gradio==4.22.0 transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.1 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=55

In [2]:

#@title Built llama-cpp-python with BLAS or CUDA automatically, wait for few minutes .
import os,torch


if torch.cuda.is_available ():
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python' % "-DLLAMA_CUBLAS=on"
else:
    cmd='CMAKE_ARGS="%s" pip install llama-cpp-python' % "-DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"

os.system (cmd)

0

In [5]:



from llama_cpp import Llama,LlamaRAMCache,LlamaDiskCache,LlamaTokenizer,LlamaState

import os,torch
from pathlib import Path
from huggingface_hub import hf_hub_download

IN_COLAB = False
Gbase="./generate/"
cache_dir="./hf/"

import torch
if torch.cuda.is_available():
    n_gpu_layers=-1
else :
    n_gpu_layers=0




description ="""<div style="font-family: Arial, sans-serif; padding: 20px;">

This showcases the use of llama-cpp-python to load various versions of 4-bit gemma 2B and 7B models, as well as a simple chat interface created with Gradio. It can be run on a mini computer or even on a phone, with satisfactory results. You are free to modify this code as you wish.
  <h2>🔗My social media links❤️</h2>

Follow <a href="https://www.facebook.com/braiml"  ytarget="_blank">🐍Brian's Page </a> if you want I share more tools .<br>
Follow<a href="https://www.facebook.com/charactersAI" target="_blank">❤️Characters AI</a>
 if you want more  videos .
<br>
  <a href="https://www.facebook.com/brian.pyai" target="_blank"> 📘facebook.com/brian.pyai</a>
 <br>
  <a href="https://www.facebook.com/braiml" target="_blank">🐍Brian's Page </a>
<br>

  <a href="https://www.facebook.com/charactersAI" target="_blank">❤️Characters AI </a>



</div>

"""
modelsPath=Gbase +"models/"

modelsPaths=["mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf",
"LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf",
"LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf",
"LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf",
"mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf",
"LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf",
"rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf",
"LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf",
"LoneStriker/zephyr-7b-gemma-sft-v0.1-GGUF/zephyr-7b-gemma-sft-v0.1-Q4_K_M.gguf",
"LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf"]


model_id="mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf" #@param ["mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/gemma-2b-GGUF/gemma-2b-Q4_K_M.gguf","LoneStriker/gemma-2b-it-GGUF/gemma-2b-it-Q4_K_M.gguf","LoneStriker/gemma-7b-GGUF/gemma-7b-Q4_K_M.gguf","LoneStriker/gemma-7b-it-GGUF/gemma-7b-it-Q4_K_M.gguf","mlabonne/Gemmalpaca-2B-GGUF/gemmalpaca-2b.Q4_K_M.gguf","LoneStriker/OrcaGemma-2B-GGUF/OrcaGemma-2B-Q4_K_M.gguf","rombodawg/EveryoneLLM-7b-Gemma-Base-GGUF/EveryoneLLM-7b-Gemma-Base-q6_k.gguf","LoneStriker/openbuddy-gemma-7b-v19.1-4k-GGUF/openbuddy-gemma-7b-v19.1-4k-Q4_K_M.gguf","LoneStriker/Gemmalpaca-7B-GGUF/Gemmalpaca-7B-Q4_K_M.gguf", "LoneStriker/zephyr-7b-gemma-v0.1-GGUF/zephyr-7b-gemma-v0.1-Q4_K_M.gguf" ,"Lewdiculous/firefly-gemma-7b-GGUF-IQ-Imatrix/firefly-gemma-7b-Q4_K_S-imatrix.gguf"]

def downHgFile(url,targetDir=modelsPath):
    fileName=Path (model_id ).name
    repo =model_id[:-len(fileName)-1]
    fileExists=os.path.exists(os.path.join(targetDir,fileName))

    print(repo,fileName,fileExists )
    if not fileExists:
        print ("Downloading file :")
        hf_hub_download(repo ,filename=fileName,local_dir=targetDir,local_dir_use_symlinks=False)

downHgFile(model_id )
modelPath=os.path.join(modelsPath,Path(model_id).name)


model = Llama(modelPath,n_gpu_layers=n_gpu_layers,n_threads=4)

tokenizer = model.tokenize




import gradio as gr
import torch

from threading import Thread




partial_message = ""
def predict(message, history):
    global partial_message
    prompt=f"""
###Below is an instruction that describes my question or task.
Write a response that appropriately completes the request:
{message}
### Response :

"""
    model_inputs = tokenizer(prompt.encode("utf-8"))
    generate_kwargs = dict(top_k=140, top_p=0.95, temp=0.95, repeat_penalty=1.1)


    partial_message = ""
    for new_token in model.generate(model_inputs,**generate_kwargs):
            #print (new_token)
            if new_token in (213,7221,1):break
            s=model.detokenize([new_token])
            #print (s)
            if isinstance(s,bytes):partial_message += s.decode("utf-8")
            yield partial_message

chatbot = gr.Chatbot(likeable=True,show_copy_button=True)
textbox = gr.Textbox (show_copy_button=True)

user="brian" #@param {type:"string"}
password="pwd" #@param {type:"string"}
auth=(user,password)

gr.ChatInterface(predict,chatbot=chatbot,stop_btn="Stop" ,retry_btn="Retry",concurrency_limit=1,description=description,multimodal=True).launch(debug=True,share=True ,inline=False,auth=auth )

mlabonne/Gemmalpaca-2B-GGUF gemmalpaca-2b.Q4_K_M.gguf False


gemmalpaca-2b.Q4_K_M.gguf:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 23 key-value pairs and 164 tensors from ./generate/models/gemmalpaca-2b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = Gemmalpaca-2B
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          gemma.block_count u32              = 18
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 8
llama_model_loader: - kv   7:              gemma.attention.head_

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a983b2fec7c5f5be8e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a983b2fec7c5f5be8e.gradio.live
